In [9]:
# Libraries

import os
import cv2
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

from sklearn.model_selection import train_test_split

import dotenv

### Hyperparameters

In [ ]:
# Load environment variables
dotenv.load_dotenv()

EPOCHS = 10
BATCH_SIZE = 32

True

### Import Dataset

In [11]:
training_dataset_path = os.path.join(os.environ['DATASET_PATH'], 'Training')

class_labels = {'glioma': 0, 'meningioma': 1, 'pituitary': 2, 'notumor': 3}
image_folders = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_labels_reverse = {v: k for k, v in class_labels.items()}


In [ ]:
# Initialize lists to store images and labels for the training dataset
images = []
labels = []

# Loop over the folders in the Training dataset to load images and their labels
for folder in image_folders:
    folder_path = os.path.join(training_dataset_path, folder)
    
    # List image files in the folder
    image_files = sorted(os.listdir(folder_path))

    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)

        # Read the image (grayscale because MRI data)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Resize image if necessary (standardize image size)
        img = cv2.resize(img, (128, 128))  # Resize to 128x128 or any size you prefer

        # Append image and label (use list append method)
        images.append(img)
        labels.append(class_labels[folder])  # Assign label based on folder name

# Convert lists to numpy arrays after appending all images and labels
images = np.array(images)
labels = np.array(labels)

# Normalize the images (optional but helps with training)
images = images.astype('float32') / 255.0

# Convert images to 3 channels (grayscale to RGB) for ResNet50
X_rgb = np.repeat(images.reshape(-1, 128, 128, 1), 3, axis=-1)

# Split the dataset into training and validation sets (80% for training, 20% for validation)
X_train, X_val, y_train, y_val = train_test_split(X_rgb, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet50 model (without the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers
base_model.trainable = False

In [13]:
# Add custom classifier layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)  # 4 classes

# Create the full model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 40,367,492 (153.99 MB)

 Trainable params: 16,779,780 (64.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train the model using the training data
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))

# Evaluate the model on the validation (split) set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 28s 175ms/step - accuracy: 0.4508 - loss: 1.2135 - val_accuracy: 0.6719 - val_loss: 0.8627
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 24s 168ms/step - accuracy: 0.6517 - loss: 0.8497 - val_accuracy: 0.7148 - val_loss: 0.7333
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 25s 173ms/step - accuracy: 0.7058 - loss: 0.7501 - val_accuracy: 0.7297 - val_loss: 0.6810
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 24s 166ms/step - accuracy: 0.7285 - loss: 0.6668 - val_accuracy: 0.7673 - val_loss: 0.6124
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 24s 165ms/step - accuracy: 0.7545 - loss: 0.6323 - val_accuracy: 0.7717 - val_loss: 0.5778
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 24s 164ms/step - accuracy: 0.7535 - loss: 0.6139 - val_accuracy: 0.7935 - val_loss: 0.5430
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 24s 164ms/step - accuracy: 0.7811 - loss: 0.5671 - val_accuracy: 0.8075 - val_loss: 0.5296
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 24s 165ms/step - accuracy: 0.7825 - loss: 0

In [16]:
def test_image(model, image_path):
    # Load and preprocess the input image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))  # Resize to match training input size
    img = img.astype('float32') / 255.0  # Normalize
    img = np.repeat(img, 3, axis=-1)  # Convert grayscale to RGB
    img = img.reshape(1, 128, 128, 3)  # Add batch dimension
    
    # Predict the class
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)  # Get the class index with the highest probability

    # Return the predicted label
    predicted_label = class_labels_reverse[predicted_class[0]]  # Map the class index to label
    return predicted_label

test_image_path1 = os.path.join(os.environ['DATASET_PATH'], 'Testing/glioma/Te-gl_0010.jpg')
test_image_path2 = os.path.join(os.environ['DATASET_PATH'], 'Testing/meningioma/Te-me_0018.jpg')
test_image_path3 = os.path.join(os.environ['DATASET_PATH'], 'Testing/notumor/Te-no_0027.jpg')
test_image_path4 = os.path.join(os.environ['DATASET_PATH'], 'Testing/pituitary/Te-pi_0094.jpg')

predicted_label1 = test_image(model, test_image_path1)
predicted_label2 = test_image(model, test_image_path2)
predicted_label3 = test_image(model, test_image_path3)
predicted_label4 = test_image(model, test_image_path4)

print(f"Predicted label: {predicted_label1}")
print(f"Predicted label: {predicted_label2}")
print(f"Predicted label: {predicted_label3}")
print(f"Predicted label: {predicted_label4}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted label: glioma
Predicted label: notumor
Predicted label: notumor
Predicted label: pituitary
